In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"status"

Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1


In [2]:
using CodeTracking
using Revise: Revise  # Revise must be loaded for CodeTracking to work

using MacroTools

In [3]:
definition(@which(1+1))

┌ Info: tracking Base
└ @ Revise /Users/oxinabox/.julia/packages/Revise/agmgx/src/Revise.jl:848


:(((x::T + y::T) where T <: BitInteger) = begin
          #= /Users/osx/buildbot/slave/package_osx64/build/base/int.jl:53 =#
          add_int(x, y)
      end)

In [4]:
definition(@which(promote(1.0, 2))) |> MacroTools.striplines |> dump

Expr
  head: Symbol function
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol promote
        2: Symbol x
        3: Symbol y
    2: Expr
      head: Symbol block
      args: Array{Any}((4,))
        1: Expr
          head: Symbol macrocall
          args: Array{Any}((2,))
            1: Symbol @_inline_meta
            2: Nothing nothing
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Expr
              head: Symbol tuple
              args: Array{Any}((2,))
                1: Symbol px
                2: Symbol py
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol _promote
                2: Symbol x
                3: Symbol y
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol not_sametype
            2: Expr
              head: Symbol tuple
              args: Array{Any}((2

## TODO
 - Need to deal with https://github.com/timholy/Revise.jl/blob/d160813bed054d80fe29854a3178fb726b3e808b/src/Revise.jl#L813

In [5]:
includet("../test/demo.jl")

In [6]:
get_method(f, args::Tuple) = get_method(f, Tuple{args...})
function get_method(f::Type{F}, args::Type{<:Tuple}) where F<:Function
    return get_method(F.instance, args)
end
get_method(f, args::Type{<:Tuple}) = Base.which(f, args)


function get_ast(meth::Method)
    ast = definition(meth)
    return MacroTools.striplines(ast)
end

get_ast (generic function with 1 method)

In [7]:
dump(:(Base.sum(1:10)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Expr
      head: Symbol .
      args: Array{Any}((2,))
        1: Symbol Base
        2: QuoteNode
          value: Symbol sum
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol :
        2: Int64 1
        3: Int64 10


In [16]:
function qualify_calls!(ast, mod)
    MacroTools.postwalk(ast) do expr
        if isexpr(expr, :call) || isexpr(expr, :macrocall)
            expr.args[1] = qualify_name(mod, expr.args[1])
        end
        expr
    end
end
qualify_name(mod::Module, name) = qualify_name(nameof(mod), name)
qualify_name(mod::Symbol, name::Symbol) = Expr(:., mod, QuoteNode(name))
qualify_name(mod, name) = name ## fallback, don't mess with things

qualify_name (generic function with 3 methods)

In [23]:
@generated function graft(f, call_args...)
    meth = get_method(f, call_args)
    
    ast = get_ast(meth)
    ast===nothing && error("no AST got")
    
    def = MacroTools.splitdef(ast)
    body = def[:body]
    body = qualify_calls!(body, meth.module)
    
    # Give all the arguments the names they have in the signature
    body = Expr(:block,
        Expr(:(=),  # give args their local names
            Expr(:tuple, def[:args]...),
            :call_args,
        ),
        body
    )
    
    # Do the actual replacement (grapfting)
    body = MacroTools.postwalk(body) do expr
        if isexpr(expr, :call)
            # TODO: make this ref `graft` right when in module
            # TODO: make this do kwargs right
            Expr(:block,
                :(println("Calling ", $(expr.args[1]))),
                Expr(:call, :graft, expr.args...)
            )
        else
            expr
        end
    end
    Core.println(sprint(dump, body)); Core.println()
    return body
end

graft (generic function with 1 method)

In [31]:
meth = @which Base._promote(1.0, 1)
ast = get_ast(meth)

:(function _promote(x::T, y::S) where {T, S}
      @_inline_meta
      R = promote_type(T, S)
      return (convert(R, x), convert(R, y))
  end)

In [30]:
def = MacroTools.splitdef(ast)

Dict{Symbol,Any} with 5 entries:
  :name        => :_promote
  :args        => Any[:(x::T), :(y::S)]
  :kwargs      => Any[]
  :body        => quote…
  :whereparams => (:T, :S)

In [33]:
meth.sparam_syms

svec(:T, :S)

In [39]:
meth.sig.body

Tuple{typeof(Base._promote),T,S} where S

In [75]:
foo(x::T) where T = string(T)
boop(x::Vector{T}) where T = string(T)

@show meth = @which boop([1])
@show meth.sparam_syms

meth = #= In[75]:4 =# @which(boop([1])) = boop(x::Array{T,1}) where T in Main at In[75]:2
meth.sparam_syms = svec(:T)


svec(:T)

In [130]:
Symbol(meth.sig.var)

:T

In [109]:
lhs = meth.sig.body

DataType

In [105]:
Base.unwrap_unionall(meth.sig)

Tuple{typeof(boop),Array{T,1}}

typeof(boop)

In [150]:
extend_with_typevars(ua::UnionAll) = UnionAll(ua.var, Tuple{ua.body.parameters..., ua.var})

function solve_type_intersect(lhs::UnionAll, rhs::Type{<:Tuple})
    num_typevars = 1
    lhs_extended = extend_with_typevars(lhs)
    rhs_extended = Tuple{rhs.parameters..., ntuple(_->Any, num_typevars)...}
    @show lhs_extended
    @show rhs_extended
    intersection::Type{<:Tuple} = typeintersect(lhs_extended, rhs_extended)
    (; Symbol(lhs.var) => intersection.parameters[end])
end

#solve_type_intersect(meth.sig, Tuple{typeof(boop), Base.typesof(([1,2])).parameters...})


function determine_type_params(meth::Method, arg_types::Type{<:Tuple})
    ftype = meth.sig.body.parameters[1]
    method_instance_sig = Tuple{ftype, arg_types.parameters...}
    return solve_type_intersect(meth.sig, method_instance_sig)
end

determine_type_params(@which(boop([1,2])), Tuple{Vector{Int}})

lhs_extended = Tuple{typeof(boop),Array{T,1},T} where T
rhs_extended = Tuple{typeof(boop),Array{Int64,1},Any}


(T = Int64,)

In [111]:
boop(x::Vector{T}) where T = string(T)
@show meth = @which boop([1])
@show meth.sig
res = typeintersect(Tuple{typeof(boop),Array{T,1}, T} where T, Tuple{typeof(boop), Vector{Int}, Any})
res.parameters[end]

meth = #= In[111]:2 =# @which(boop([1])) = boop(x::Array{T,1}) where T in Main at In[111]:1
meth.sig = Tuple{typeof(boop),Array{T,1}} where T


Int64

In [128]:
res

Tuple{typeof(boop),Array{Int64,1},Int64}

In [124]:
UnionAll(meth.sig.var, meth.sig.body) == meth.sig

true

In [120]:
meth.sig.var |> typeof

TypeVar

In [66]:
minst.invoke()

MethodError: MethodError: objects of type Ptr{Nothing} are not callable

In [25]:
graft(promote, 1.0, 1)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Expr
          head: Symbol tuple
          args: Array{Any}((2,))
            1: Symbol x
            2: Symbol y
        2: Symbol call_args
    2: Expr
      head: Symbol block
      args: Array{Any}((4,))
        1: Expr
          head: Symbol macrocall
          args: Array{Any}((2,))
            1: Expr
              head: Symbol .
              args: Array{Any}((2,))
                1: Symbol Base
                2: QuoteNode
            2: Nothing nothing
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Expr
              head: Symbol tuple
              args: Array{Any}((2,))
                1: Symbol px
                2: Symbol py
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: Expr
                2: Expr
        3: Expr
          head: Symbol block
   

UndefVarError: UndefVarError: T not defined

In [42]:
?Base.@_inline_meta

No documentation found.

`Base.@_inline_meta` is a macro.

```
# 1 method for macro "@_inline_meta":
[1] @_inline_meta(__source__::LineNumberNode, __module__::Module) in Base at essentials.jl:15
```


In [8]:
Base.which(+, Tuple{Int,Int})

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

Expr
  head: Symbol .
  args: Array{Any}((2,))
    1: Expr
      head: Symbol .
      args: Array{Any}((2,))
        1: Symbol Base
        2: QuoteNode
          value: Symbol Sys
    2: QuoteNode
      value: Symbol ARCH


In [10]:
@less Base.which(typeof(+), Tuple{Int})

# This file is a part of Julia. License is MIT: https://julialang.org/license

# name and module reflection

"""
    nameof(m::Module) -> Symbol

Get the name of a `Module` as a `Symbol`.

# Examples
```jldoctest
julia> nameof(Base.Broadcast)
:Broadcast
```
"""
nameof(m::Module) = ccall(:jl_module_name, Ref{Symbol}, (Any,), m)

"""
    parentmodule(m::Module) -> Module

Get a module's enclosing `Module`. `Main` is its own parent.

# Examples
```jldoctest
julia> parentmodule(Main)
Main

julia> parentmodule(Base.Broadcast)
Base
```
"""
parentmodule(m::Module) = ccall(:jl_module_parent, Ref{Module}, (Any,), m)

"""
    moduleroot(m::Module) -> Module

Find the root module of a given module. This is the first module in the chain of
parent modules of `m` which is either a registered root module or which is its
own parent module.
"""
function moduleroot(m::Module)
    while true
        is_root_module(m) && return m
        p = parentmodule(m)
        p == m && return m
        m = p
    end

In [11]:
@generated function testem1(args...)
    true_args = Any[:x, :y]

end

testem1(2,37) |> dump

Array{Any}((2,))
  1: Symbol x
  2: Symbol y


In [12]:
quote
    println("Calling ", 1)
    foo()
    end |> dump

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[12]
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol println
        2: String "Calling "
        3: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[12]
    4: Expr
      head: Symbol call
      args: Array{Any}((1,))
        1: Symbol foo
